In [1]:
filtered_arr = [0 if x == -1 else 1 for x in [-1, 1] if x != 0]
print(filtered_arr)
binary_str = ''.join(map(str, filtered_arr))
print(binary_str)
# 取到二进制表示的数
map_num = int("11110", 9)
print(map_num)

[0, 1]
01
7380


In [2]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [3]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))
    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) @ np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) @ np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])
        hamiltonian += term1 + term2
    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [5]:
import sys
sys.path.append('../')
import z_library.linear_system as ls
Cons = np.array([[-1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
              [-1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
              [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])
u = ls.find_basic_solution(Cons)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd = calculate_hamiltonian(v, w)

In [6]:
import sys
sys.path.append('../')
import z_library.linear_system as ls
Cons = np.array([[-1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
              [-1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
              [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])
u = ls.find_basic_solution(Cons)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd2 = calculate_hamiltonian(v, w)

In [7]:
np.where(Hd != Hd2)


(array([], dtype=int64), array([], dtype=int64))

In [8]:
from scipy.linalg import expm

ImportError: cannot import name 'expm' from 'scipy' (/home/jiangqifan/Tools/anaconda3/envs/FLP/lib/python3.8/site-packages/scipy/__init__.py)

In [ ]:
h = np.array([[ 3,    0],
              [   0, -3]])

In [ ]:
print(expm(-1j * (2 * np.pi) * h))

[[-1.-0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [ ]:
print(expm(-1j * (1) * h))
print(expm(-1j * (1 + 2 * np.pi) * h))

[[-1.-0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
[[-1.-0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [ ]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params)
  qc = QuantumCircuit(num_qubits)
  gamma = params
  for i in [1,3,4,6,8,9]:
    qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * (1.5 + 3) * Hd2), range(num_qubits))
  qc.measure_all()
  return qc

qc_res = build_circ(params=[23])
backend = Aer.get_backend('aer_simulator')
shots=100000
backend.shots = 100000
counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
# plot_histogram(counts)
sorted_counts = sorted(counts, key=counts.get, reverse=True)
print("\n----------------- Full result ---------------------")
print("selection\t\tprobability\tvalue")  
print("---------------------------------------------------")
for x in sorted_counts[:20]:
  print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))


----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1101101001 59.94% 6
1110100101 24.05% 5
1101011010 8.43% 7
1010100000 4.83% 4
1110010110 1.83% 6
0101010000 0.92% 5



----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1110010110 41.88% 6
1010100000 22.63% 4
1101011010 15.31% 7
1101101001 9.29% 6
0101010000 8.82% 5
1110100101 2.07% 5


In [ ]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [ ]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))

In [ ]:
sum([ 1.042e+00,  1.031e+00,  1.054e+00,  1.022e+00,  1.045e+00])

5.194

In [ ]:
dep = 1
print(f'depth == {dep}')
test([5.194 + 2*np.pi])

depth == 1
Iteration 10, Result: 5.242
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.242
       x: [ 1.138e+01]
    nfev: 13
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 73.50% 5
1110010110 16.03% 6
1101101001 8.67% 6
1010100000 0.88% 4
1110100101 0.85% 5
1101011010 0.07% 7


In [ ]:
dep = 2
print(f'depth == {dep}')
test(np.full(dep, np.pi/3))

depth == 2
Iteration 10, Result: 4.921
Iteration 20, Result: 4.899
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.899
       x: [ 2.772e+00  3.582e-01]
    nfev: 26
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 38.43% 5
1110100101 26.24% 5
1010100000 23.83% 4
1101101001 7.38% 6
1110010110 3.19% 6
1101011010 0.92% 7


In [ ]:
dep = 3
print(f'depth == {dep}')
test(np.full(dep, np.pi/3))

depth == 3
Iteration 10, Result: 4.9185
Iteration 20, Result: 4.901
Iteration 30, Result: 4.899
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.899
       x: [ 1.051e+00  1.040e+00  1.040e+00]
    nfev: 31
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 38.43% 5
1110100101 26.24% 5
1010100000 23.84% 4
1101101001 7.38% 6
1110010110 3.19% 6
1101011010 0.92% 7


In [ ]:
dep = 4
print(f'depth == {dep}')
test(np.full(dep, np.pi/3))

depth == 4
Iteration 10, Result: 6.0135
Iteration 20, Result: 4.804
Iteration 30, Result: 4.8045
Iteration 40, Result: 4.8035
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.8035
       x: [ 2.048e+00  1.044e+00  1.054e+00  1.047e+00]
    nfev: 40
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1110100101 61.33% 5
1010100000 29.37% 4
1110010110 7.60% 6
1101011010 1.24% 7
0101010000 0.46% 5


In [ ]:
dep = 5
print(f'depth == {dep}')
test(np.full(dep, np.pi/3))

depth == 5
Iteration 10, Result: 4.925
Iteration 20, Result: 4.8045
Iteration 30, Result: 4.8035
Iteration 40, Result: 4.8035
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.8035
       x: [ 1.042e+00  1.031e+00  1.054e+00  1.022e+00  1.045e+00]
    nfev: 46
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1110100101 61.33% 5
1010100000 29.36% 4
1110010110 7.61% 6
1101011010 1.24% 7
0101010000 0.46% 5
1101101001 0.00% 6


In [ ]:
dep = 6
print(f'depth == {dep}')
test(np.full(dep * 2, np.pi/3))

depth == 6
Iteration 10, Result: 6.0285
Iteration 20, Result: 4.508
Iteration 30, Result: 5.6785
Iteration 40, Result: 4.3685
Iteration 50, Result: 4.334
Iteration 60, Result: 4.246
Iteration 70, Result: 4.154
Iteration 80, Result: 4.098
Iteration 90, Result: 4.0885
Iteration 100, Result: 4.063
Iteration 110, Result: 4.0505
Iteration 120, Result: 4.0475
Iteration 130, Result: 4.0485
Iteration 140, Result: 4.0455
Iteration 150, Result: 4.05
Iteration 160, Result: 4.0465
Iteration 170, Result: 4.045
Iteration 180, Result: 4.045
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.045
       x: [ 1.097e+00  1.081e+00  1.407e+00  1.353e+00  2.052e+00
            1.199e+00  9.227e-01  5.431e-01  1.797e+00  9.038e-01
            8.691e-01  9.233e-01]
    nfev: 185
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 96.19% 4
0101010000 2.16

In [ ]:
dep = 7
print(f'depth == {dep}')
test(np.full(dep * 2, np.pi/3))

depth == 7
Iteration 10, Result: 6.029
Iteration 20, Result: 4.724
Iteration 30, Result: 4.7375
Iteration 40, Result: 4.55
Iteration 50, Result: 4.303
Iteration 60, Result: 4.299
Iteration 70, Result: 4.2595
Iteration 80, Result: 4.2375
Iteration 90, Result: 4.22
Iteration 100, Result: 4.1835
Iteration 110, Result: 4.185
Iteration 120, Result: 4.1745
Iteration 130, Result: 4.175
Iteration 140, Result: 4.1715
Iteration 150, Result: 4.1725
Iteration 160, Result: 4.17
Iteration 170, Result: 4.17
Iteration 180, Result: 4.17
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.17
       x: [ 1.108e+00  1.951e+00 ...  1.015e+00  9.465e-01]
    nfev: 187
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 87.36% 4
1110100101 9.37% 5
1110010110 1.70% 6
0101010000 0.90% 5
1101011010 0.61% 7
1101101001 0.06% 6


In [ ]:
dep = 8
print(f'depth == {dep}')
test(np.full(dep * 2, np.pi/3))

depth == 8
Iteration 10, Result: 5.481
Iteration 20, Result: 4.7265
Iteration 30, Result: 4.437
Iteration 40, Result: 4.4315
Iteration 50, Result: 4.5425
Iteration 60, Result: 4.292
Iteration 70, Result: 4.228
Iteration 80, Result: 4.184
Iteration 90, Result: 4.199
Iteration 100, Result: 4.1635
Iteration 110, Result: 4.1255
Iteration 120, Result: 4.1155
Iteration 130, Result: 4.102
Iteration 140, Result: 4.0925
Iteration 150, Result: 4.0845
Iteration 160, Result: 4.0745
Iteration 170, Result: 4.068
Iteration 180, Result: 4.0625
Iteration 190, Result: 4.056
Iteration 200, Result: 4.055
Iteration 210, Result: 4.054
Iteration 220, Result: 4.0525
Iteration 230, Result: 4.052
Iteration 240, Result: 4.051
Iteration 250, Result: 4.051
Iteration 260, Result: 4.051
Iteration 270, Result: 4.0505
Iteration 280, Result: 4.0505
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.0505
       x: [ 1.056e+00  1.390e+00 ...  5.169e-01  8.260e-01]
    nfe

In [ ]:
dep = 9
print(f'depth == {dep}')
test(np.full(dep * 2, np.pi/3))

depth == 9
Iteration 10, Result: 5.404
Iteration 20, Result: 5.0355
Iteration 30, Result: 5.1075
Iteration 40, Result: 5.1065
Iteration 50, Result: 4.612
Iteration 60, Result: 4.6225
Iteration 70, Result: 4.4235
Iteration 80, Result: 4.2495
Iteration 90, Result: 4.162
Iteration 100, Result: 4.077
Iteration 110, Result: 4.066
Iteration 120, Result: 4.0545
Iteration 130, Result: 4.0555
Iteration 140, Result: 4.055
Iteration 150, Result: 4.0525
Iteration 160, Result: 4.0525
Iteration 170, Result: 4.0545
Iteration 180, Result: 4.053
Iteration 190, Result: 4.0515
Iteration 200, Result: 4.0515
Iteration 210, Result: 4.0515
Iteration 220, Result: 4.0515
Iteration 230, Result: 4.0515
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.0515
       x: [ 4.839e-01  2.106e+00 ...  6.650e-01  1.185e+00]
    nfev: 231
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------